# CHL Curve Conrady Prediction

This notebook uses the 3-term Conrady model to predict the longitudinal chromatic aberration (CHL) curve based on user-defined optical and sampling conditions. The `achromatcfw.core.cfw` module is then used for real-time evaluation of color fringes through focus.

In [1]:
# Add ../src to the Python search path
import sys
from pathlib import Path
sys.path.append(str(Path('..').resolve() / 'src'))
from achromatcfw.core.cfw import Farbsaumbreite

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, FloatSlider, Checkbox, interact, Output
from IPython.display import display, clear_output

In [2]:
# ---------- 0. λ-grid -----------------------------------------------------
# lam_f is the master wavelength grid, units: nanometres (nm), 1 nm step
lam_f = np.linspace(400, 700, 301)  # nm

# CHLdata_global will later hold a sampled longitudinal chromatic focal shift Δz curve
CHLdata_global = None

out = Output()

# ---------- 1. Glass library (Conrady in nm-domain) -----------------------
# Conrady dispersion coefficients converted to the nm domain
mm_to_um = 1e3        # millimetres → micrometres
µm_to_nm = 1e3        # micrometres → nanometres
µm_to_nm35 = 1e3**3.5 # micrometres → nanometres^(3.5)

glass_db = {
    "N-BK7":  dict(a=0.497482, B=1.010751e-2 * µm_to_nm,  C=3.288943e-4 * µm_to_nm35),
    "N-F2":   dict(a=0.586679, B=1.554256e-2 * µm_to_nm,  C=1.075652e-3 * µm_to_nm35),
    "N-BAK4": dict(a=0.545866, B=1.165667e-2 * µm_to_nm,  C=4.858270e-4 * µm_to_nm35),
    "N-SF5":  dict(a=0.633188, B=1.797255e-2 * µm_to_nm,  C=1.389075e-3 * µm_to_nm35),
    "N-SF6":  dict(a=0.749198, B=2.409297e-2 * µm_to_nm,  C=2.328573e-3 * µm_to_nm35),
    "N-BAF10": dict(a=0.63934, B=1.5177e-2 * µm_to_nm, C=7.54e-4  * µm_to_nm35,),
    "SF10": dict(a=0.68078, B=2.1250e-2 * µm_to_nm, C=1.7590e-3 * µm_to_nm35,),
}

# ---------- 2. Widgets ----------------------------------------------------
# Slider values and labels are shown in their respective physical units

g1_dd = Dropdown(options=list(glass_db), value="N-BK7", description="Glass 1")
g2_dd = Dropdown(options=list(glass_db), value="N-F2",  description="Glass 2")

phi_sl = FloatSlider(
    min=-0.05, max=0.05, step=1e-4, value=+0.010,
    description="Φ₀ [mm⁻¹]", continuous_update=False,
    readout_format=".4f", layout={"width": "380px"},
)

lam1_sl = FloatSlider(
    min=400, max=700, step=1,
    value=486.1, description="λ₁ [nm]", continuous_update=False,
)
lam2_sl = FloatSlider(
    min=400, max=700, step=1,
    value=656.3, description="λ₂ [nm]", continuous_update=False,
)

lam0_sl = FloatSlider(
    min=400, max=700, step=1,
    value=587.3, description="λ₀ [nm]", continuous_update=False,
    layout={"width": "380px"},
)

auto_chk = Checkbox(value=True, description="Auto‑refocus @ λ₀")

# ---------- 3. Conrady helper --------------------------------------------

def conrady(lam_nm: float, A: float, B: float, C: float) -> np.ndarray:
    """Conrady dispersion formula (λ in nm, Δz in µm):
    Δz = A + B/λ + C/λ³·⁵
    """
    lam_nm = np.asarray(lam_nm, dtype=np.float64)
    return A + B / lam_nm + C / lam_nm**3.5

# ---------- 4. Callback ---------------------------------------------------

def update(g1, g2, Φ0, λ1, λ2, λ0, auto_refocus):
    """Interactive callback: computes focal‑shift curve, plots dispersion components,
    and stores a down‑sampled Δz curve to CHLdata_global.
    All printed values are reported in explicit physical units.
    """
    global CHLdata_global
    with out:
        clear_output(wait=True)

        # Guard: λ1 and λ2 must be distinct
        if abs(λ2 - λ1) < 1e-6:
            print("λ₁ and λ₂ must differ.")
            return

        # -- glass coefficients -------------------------------------------
        a1, B1, C1 = (glass_db[g1][k] for k in ("a", "B", "C"))
        a2, B2, C2 = (glass_db[g2][k] for k in ("a", "B", "C"))

        # -- two‑colour achromat condition: solve r = K₂ / K₁ --------------
        Δinv = 1 / λ1 - 1 / λ2
        Δinv35 = 1 / λ1**3.5 - 1 / λ2**3.5
        num = B1 * Δinv + C1 * Δinv35
        den = B2 * Δinv + C2 * Δinv35
        if abs(den) < 1e-12:
            print("The selected glass pair cannot be achromatised for the given wavelengths.")
            return
        r = -num / den  # ratio of the two bending powers

        # -- solve K₁, K₂ from Φ₀ = K₁·a₁ + K₂·a₂ -------------------------
        if abs(a1 + r * a2) < 1e-12:
            print("Division by zero in Φ₀ equation—adjust Φ₀ or choose different glasses.")
            return
        K1 = Φ0 / (a1 + r * a2)
        K2 = r * K1

        # -- longitudinal chromatic focal shift ----------------------------
        α, β = K1 * B1 + K2 * B2, K1 * C1 + K2 * C2
        if abs(Φ0) < 1e-9:
            print("Φ₀ is too close to zero—use a non‑zero value.")
            return
        # coefficients for Δz (µm): A [µm], B [µm·nm], C [µm·nm³·⁵]
        A, B, C = 1 / Φ0, -α / Φ0**2, -β / Φ0**2
        A *= mm_to_um  # mm → µm
        B *= mm_to_um
        C *= mm_to_um
        dz = conrady(lam_f, A, B, C)  # µm

        # -- automatic refocus at λ0 --------------------------------------
        if auto_refocus:
            dz0 = conrady(λ0, A, B, C)
            dz -= dz0  # µm (shifted)

        # -- sample Δz every 10 nm ----------------------------------------
        lam_sample = np.arange(400, 701, 10, dtype=np.float64)  # nm
        dz_sample = conrady(lam_sample, A, B, C)  # µm
        if auto_refocus:
            dz_sample -= dz0
        CHLdata_global = np.vstack([lam_sample, dz_sample]).T  # shape (31, 2)

        # -- plotting ------------------------------------------------------
        fig, (axλ, axcmp) = plt.subplots(1, 2, figsize=(12, 5), constrained_layout=True)

        # focal shift curve
        axλ.plot(lam_f, dz, "r", lw=2, label="Δz(λ)")
        for lam_val, col in [(λ1, "k"), (λ2, "k")]:
            axλ.axvline(lam_val, color=col, ls=":", lw=0.6)
        if auto_refocus:
            axλ.axvline(λ0, color="m", ls="--", lw=0.6, label="λ₀")
        axλ.set(xlabel="λ [nm]", ylabel="Δz [µm]", xlim=(400, 700))
        axλ.grid(lw=0.3)
        axλ.legend()

        # dispersion components
        axcmp.plot(lam_f, B / lam_f, "g", label="B / λ")
        axcmp.plot(lam_f, C / lam_f**3.5, "b", label="C / λ³·⁵")
        axcmp.set(xlabel="λ [nm]", ylabel="Component magnitude [µm]", xlim=(400, 700))
        axcmp.grid(lw=0.3)
        axcmp.legend(fontsize=8)

        # figure title
        tag = f" (refocus @ {λ0:.0f} nm)" if auto_refocus else ""
        fig.suptitle(
            f"{g1} + {g2}   Φ₀ = {Φ0:+.4f} mm⁻¹   K₁ = {K1:+.4f}, K₂ = {K2:+.4f} (r = {r:+.4f}){tag}",
            fontsize=12,
        )
        plt.show()
        plt.close(fig)

        # -- console read‑out ---------------------------------------------
        print(f"r = K₂ / K₁ = {r:+.4f}")
        print(f"A = {A:+.2f} µm   B = {B:+.2e} µm·nm   C = {C:+.2e} µm·nm³·⁵")
        print(f"α = {α:+.3e}   β = {β:+.3e}")
        print("CHLdata_global updated → shape", CHLdata_global.shape)

# ---------- 5. Launch -----------------------------------------------------

interact(
    update,
    g1=g1_dd,
    g2=g2_dd,
    Φ0=phi_sl,
    λ1=lam1_sl,
    λ2=lam2_sl,
    λ0=lam0_sl,
    auto_refocus=auto_chk,
)

display(out)


interactive(children=(Dropdown(description='Glass\xa01', options=('N-BK7', 'N-F2', 'N-BAK4', 'N-SF5', 'N-SF6',…

Output()

In [3]:
# ---------------------------------------------------------------------
# Constants (edit here, everything else updates automatically)
# ---------------------------------------------------------------------
K: float            = 2.2   # f‑number
F_VALUE: float      = 8.0   # default exposure‑curve factor
GAMMA_VALUE: float  = 1.0   # default gamma

TOL: float          = 0.15  # colour‑difference tolerance

XRANGE      = 400        # x window half width (µm)

defocusrange: int   = 1500  # ± defocus sweep range (µm)
defocus_step: int   = 10    # defocus sampling step (µm)

z_vals = np.arange(-defocusrange,
                   defocusrange + defocus_step,
                   defocus_step, dtype=float)


In [4]:
threshold = 1e-6

widths = np.array([Farbsaumbreite(z, CHLdata=CHLdata_global[:, 1]) for z in z_vals])


zero_crossings = []
for i in range(len(widths) - 1):
    w0, w1 = widths[i], widths[i + 1]
    if w0 <= threshold and w1 > threshold:
        z0 = np.interp(0.0, [w0, w1], [z_vals[i], z_vals[i + 1]])
        zero_crossings.append(z0)
    elif w0 > threshold and w1 <= threshold:
        z0 = np.interp(0.0, [w0, w1], [z_vals[i], z_vals[i + 1]])
        zero_crossings.append(z0)

if len(zero_crossings) >= 2:
    z0_left, z0_right = zero_crossings[0], zero_crossings[-1]

    mask = (z_vals >= z0_left) & (z_vals <= z0_right)
    mean_valid = widths[mask].mean()

    print(f"Max  width: {widths.max():.2f}")
    print(f"Mean width (between first & last zero): {mean_valid:.2f}")
    print(f"First zero at  z ≈ {z0_left:.3f}    " f"Last zero at  z ≈ {z0_right:.3f}")
    print("To make the computation faster, use a narrower defocus range.")

else:
    print("Please enlarge the defocus range to find zero crossings.")


Max  width: 117.00
Mean width (between first & last zero): 84.88
First zero at  z ≈ -900.000    Last zero at  z ≈ 990.000
To make the computation faster, use a narrower defocus range.


### 📘 Explanation of the Interactive Achromat Focal Shift Simulator

This interactive simulation models the **longitudinal chromatic focal shift (Δz)** of a classical achromatic doublet based on **Conrady's dispersion formula** and paraxial optics.

---

#### 🔧 Workflow Breakdown

1. **Wavelength Grid**  
   The simulation uses a master wavelength grid `lam_f` ranging from 400 nm to 700 nm in 1 nm steps.

2. **Glass Library**  
   Each glass is characterized by three parameters in the Conrady dispersion formula:
   $$
   n(\lambda) = 1 + a_i + \frac{b_i}{\lambda} + \frac{c_i}{\lambda^{3.5}}
   $$
   where $ \lambda $ is in nanometers. The coefficients $ b_i $ and $ c_i $ are converted from their original units into nanometer domain for consistency:
   - $ B $: µm·nm
   - $ C $: µm·nm³⋅⁵

3. **Conrady Model for Focal Shift**  
   According to our derivation, the focal shift Δz (in µm) is modeled as:
   $$
   \Delta z(\lambda) = A + \frac{B}{\lambda} + \frac{C}{\lambda^{3.5}}
   $$
   where:
   $$
   A = \frac{1}{\Phi_0}, \quad
   B = -\frac{\alpha}{\Phi_0^2}, \quad
   C = -\frac{\beta}{\Phi_0^2}
   $$
   and:
   $$
   \alpha = K_1 B_1 + K_2 B_2, \quad
   \beta = K_1 C_1 + K_2 C_2
   $$

4. **Achromat Condition**  
   To ensure the doublet is **achromatic at two wavelengths** $ \lambda_1 $ and $ \lambda_2 $, we impose:
   $$
   K_1 B_1 + K_2 B_2 = 0, \quad
   K_1 C_1 + K_2 C_2 = 0
   $$
   This leads to the ratio:
   $$
   r = \frac{K_2}{K_1} = -\frac{B_1 \Delta(1/\lambda) + C_1 \Delta(1/\lambda^{3.5})}{B_2 \Delta(1/\lambda) + C_2 \Delta(1/\lambda^{3.5})}
   $$

5. **Doublet Power Equation**  
   With known design power $ \Phi_0 $ and ratio $ r = K_2 / K_1 $, the individual element powers are:
   $$
   \Phi_0 = K_1 a_1 + K_2 a_2 = K_1 (a_1 + r a_2)
   \Rightarrow
   K_1 = \frac{\Phi_0}{a_1 + r a_2}, \quad
   K_2 = r K_1
   $$

6. **Refocus to Design Wavelength**  
   The Δz curve can be **refocused** to ensure zero defocus at the primary design wavelength $ \lambda_0 $:
   $$
   \Delta z(\lambda) \gets \Delta z(\lambda) - \Delta z(\lambda_0)
   $$

7. **Degree of Freedom**  
   With the given $(a_i, b_i, c_i)$ for each glass, the design focal length $ \Phi_0 $, and the achromatism condition imposed, the system has **zero degrees of freedom** left in the paraxial model.

   However, the simulator provides sliders for:
   - $ \lambda_1 $, $ \lambda_2 $: the achromatism wavelengths
   - $ \lambda_0 $: the design wavelength

   This allows users to explore how **different design requirements** affect the predicted focal shift curve $ \Delta z(\lambda) $. These curves can then be evaluated in real-time using the **CFW toolbox**, which computes **color fringe width** through focus, offering a perceptual metric for the quality of achromatic correction.

---
